# Import Dependencies

In [116]:
import numpy as np
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from collections import defaultdict
import json

# Load Environments

In [47]:
load_dotenv()

True

# OpenAI API Configuration

In [48]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

In [49]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response

# Convert Multiple Excel sheets to CSV

In [49]:
excel_file = '../data/Moonshot Tracker Results - Auto.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv(f"../data/{sheet_name}.csv", index=False)

In [50]:
projects_raw = pd.read_csv("../data/Projects.csv")
outputs = pd.read_csv("../data/Outputs.csv")

# Data Relationship

![image info](../data/data_relationship.jpeg)

In [51]:
projects_raw.columns

Index(['Project ID', 'Project Title', 'Budget', 'Country', 'Country Code',
       'Link', 'Donors', 'VF or Non-VF', 'Output Count',
       'Direct Beneficiaries', 'GHG Emissions Reduction', 'description',
       'sdgs', 'solution', 'Gender Marker'],
      dtype='object')

In [52]:
project_target_column  = ['Project ID', 'Project Title', 'Budget', 'VF or Non-VF', 'Output Count',
       'Direct Beneficiaries']

In [53]:
projects_raw = projects_raw[project_target_column]

In [54]:
outputs.columns

Index(['Project ID', 'Title', 'Link', 'Budget', 'Beneficiary Category',
       'Indicator', 'Baseline', 'Target', 'Notes', 'Donors',
       'Gender (% female)', 'VF or Non-VF', 'Tag', 'SEH Taxonomy',
       'RISE Taxonomy', 'Flagship', 'Technology', 'Output ID', 'Country Code',
       'Category', 'name 2', 'm49', 'continent-region', 'sub-region',
       'sids-region', 'un-member', 'undp-sids', 'un-region', 'Country Name',
       'Region', 'Economy', 'LDC', 'SIDS', 'LLDC', 'HDI', 'Status',
       'Direct Conversion Factor', 'Direct Beneficiaries', 'Output Category',
       'Beneficiary Category Pre', 'Description'],
      dtype='object')

In [55]:
outputs_target_column  = ['Project ID', 'Beneficiary Category', 'continent-region', 'Country Name',
       'Region', 'Direct Beneficiaries']

In [106]:
outputs = outputs[outputs_target_column]
outputs = outputs.replace({np.nan: None})

# Functions for Extracting Information
Our Goal: Generate text summaries of the UNDP portfolio for each country

Approach: 
1. Assign 'continent-region', 'Country Name', and 'Region' to the project dataframe.
2. Create a function to get each 'project' for the specified 'continent-region', 'Country Name', or 'Region'.
3. Create a function to get the 'output's of each 'project'.
4. Create a function to call the OpenAI API and generate a summary for the UNDP portfolio for each country using a suitable prompt.

## 1. Assign 'continent-region', 'Country Name', and 'Region' to the project dataframe

In [10]:
output_country = outputs[['Project ID', 'continent-region', 'Country Name','Region']]

In [11]:
output_country = output_country.drop_duplicates().reset_index(drop=True)

In [12]:
projects = pd.merge(projects_raw, output_country, on = 'Project ID')

In [13]:
projects.rename(columns = {'continent-region':'continent', 'Country Name': 'country','Region': 'undp_region'}, inplace = True)

In [107]:
projects = projects.replace({np.nan: None})

In [108]:
projects.head()

,Project ID,Project Title,Budget,Donors,VF or Non-VF,Output Count,Direct Beneficiaries,description,continent,country,undp_region
0,117913,Résilience des communautés et des écosystèmes,3459600.0,['UNITED NATIONS DEVELOPMENT PRO'],Non-VF,3,3697.5,Il s’agit de la promotion du développement Loc...,Africa,Togo,RBA
1,134793,Accès aux énergies renouvelables en milieu rur...,1431552.0,['UNITED NATIONS DEVELOPMENT PRO'],Non-VF,3,31525.0,Accélération de l’accès aux énergies renouvela...,Africa,Togo,RBA
2,91204,Apoyo a la Modernización de La Gestión Ambiental,4202031.0,UNDP (TRAC 4000) CLIMATE PROMISE (28708) - NDC...,Non-VF,1,0.0,None,Americas,Panama,RBLAC
3,133871,Beyond Recovery COVID19 Energy,473000.0,['UNITED NATIONS DEVELOPMENT PRO'],Non-VF,2,17500.0,"Acceso universal, a través de la implementació...",Americas,Panama,RBLAC
4,6613,Africa Mini-grids Program,1363947.0,None,VF,3,4936.0,None,Africa,Zambia,RBA


In [109]:
projects.shape

(340, 11)

In [110]:
projects['Project ID'].nunique()

339

## 2. Create a function to get each 'project' for the specified 'continent-region', 'Country Name', or 'Region'

In [17]:
def get_project_per_region(region_type, region_value):
    target_projects = None
    region_type, region_value = region_type.lower(), region_value.lower()
    if region_type == 'continent':
        target_projects = projects[projects['continent'].str.lower().isin([region_value])]
    elif region_type == 'country':
        target_projects = projects[projects['country'].str.lower().isin([region_value])]
    elif region_type == 'undp_region':
        target_projects = projects[projects['undp_region'].str.lower().isin([region_value])]
    return target_projects

In [18]:
# test function
print(get_project_per_region('continent', 'Africa')['continent'].unique())
print(get_project_per_region('continent', 'americas')['continent'].unique())
print(get_project_per_region('country', 'panama')['country'].unique())
print(get_project_per_region('undp_region', 'rblac')['undp_region'].unique())

['Africa']
['Americas']
['Panama']
['RBLAC']


## 3. Create a function to get the 'output's of each 'project'

In [80]:
outputs.head()

,Project ID,Beneficiary Category,continent-region,Country Name,Region,Direct Beneficiaries
0,117913,Clean Electricity,Africa,Togo,RBA,3697.5
1,117913,Other,Africa,Togo,RBA,0.0
2,117913,Other,Africa,Togo,RBA,0.0
3,134793,Clean Electricity,Africa,Togo,RBA,6525.0
4,134793,Other,Africa,Togo,RBA,0.0


In [90]:
def get_output_per_project(project_ids):
    return outputs[outputs['Project ID'].isin(project_ids)]

In [91]:
# test function
project_ids = ['6613', 'missing-ZMB-1', '1061012', '0']
get_output_per_project(project_ids)

,Project ID,Beneficiary Category,continent-region,Country Name,Region,Direct Beneficiaries
9,6613,Agriculture and Food System,Africa,Zambia,RBA,4936.0
10,6613,Other,Africa,Zambia,RBA,0.0
11,6613,Clean Electricity,Africa,Zambia,RBA,0.0
12,missing-ZMB-1,Health Services,Africa,Zambia,RBA,810000.0
13,missing-ZMB-1,Other,Africa,Zambia,RBA,0.0
391,1061012,Other,Asia,Viet Nam,RBAP,0.0
392,1061012,Unknown,Asia,Viet Nam,RBAP,73500.0
393,1061012,Unknown,Asia,Viet Nam,RBAP,570.0
394,1061012,Other,Asia,Viet Nam,RBAP,0.0
395,1061012,Other,Asia,Viet Nam,RBAP,0.0


## 4. Create a function to call the OpenAI API and generate a summary for the UNDP portfolio for each country using a suitable prompt.

Example Outputs

RBAP (regional bureau of asia pacific):

The UNDP portfolio of energy-related projects in RBAP includes 83 projects across 23 countries during the 2022-2025 Strategic Plan. The total budget of these projects is 422 million USD. The total direct beneficiaries target by these projects include 16.3 million people. This include 7.8 million from VF projects and 8.5 from Non-VF. 4.3 million of these beneficiaries are from productive use of energy, which includes 1.7 million from health services, 1.2 million from energy infrastructures services, and 400,000 from agricultural and food systems. Many projects also include outputs related to support to policy or regulatory frameworks or other system benefits that provide indirect benefits.

RBA (Africa):

The UNDP portfolio in RBA during the period of 2022-2025 Strategic Plan includes 112 energy-related projects active across 41 countries. The total budget of these projects is 1.44 billion USD. The direct beneficiaries targeted by these projects include 54.9 million people. This includes 6.5 million from VF projects and 48.4 million from Non-VF. Indirect beneficiaries are supported through support to policy or regulatory frameworks, clean energy or efficiency campaigns, or other system changes that provide indirect benefits.
 
Of those directly benefiting, 17.1 million of these beneficiaries include direct access to clean electricity. Many other of the 54.9M direct beneficiaries benefit from productive uses of energy including 19.2 million from health services, 12.1 million from water services, 1.0 million from clean cooking, and 10 million additional beneficiaries from other categories including energy infrastructure services, transportation and e-mobility, renewable energy for waste management, and market development.

RBEC:

The UNDP portfolio of energy-related projects in RBEC includes 63 projects across 18 countries during the 2022-2025 Strategic Plan. The total budget of these projects is 427 million USD. The total direct beneficiaries target by these projects include 7.1 million people. This include 1.4 million from VF projects and 5.7 million from Non-VF. 4.0 million of these beneficiaries are from access to electricity and the other 3.1 million include beneficiaries from market development, energy infrastructure services, transportation and e-mobility, renewable energy for waste management, and other productive uses of energy. Many projects also include outputs related to support to policy or regulatory frameworks, clean energy or efficiency campaigns, or other system changes that provide indirect benefits.

In [111]:
# continent-region':'continent', 'Country Name': 'country','Region': 'undp_region'
continent_vals = projects['continent'].unique()
country_vals = projects['country'].unique()
undp_region_vals = projects['undp_region'].unique()

print(f"continent values: {continent_vals}")
print(f"country values: {country_vals}")
print(f"undp region values: {undp_region_vals}")

continent values: ['Africa' 'Americas' 'Oceania' 'Asia' 'Europe' None]
country values: ['Togo' 'Panama' 'Zambia' 'Malawi' 'South Africa' 'Samoa' 'Lebanon'
 'Brazil' 'Jamaica' 'Algeria' 'Yemen' 'Belize' 'Iran' 'Colombia' 'Comoros'
 'Azerbaijan' 'Moldova Republic' 'Congo' 'Jordan' 'Dominican Republic'
 'Sierra Leone' 'Saudi Arabia' 'Senegal' 'Suriname' 'Gambia' 'Uganda'
 'Mexico' 'Botswana' 'Guyana' 'Ethiopia' 'Belarus' 'Bahrain'
 'Papua New Guinea' 'Madagascar' 'Venezuela' 'Lesotho' 'Zimbabwe'
 'Ecuador' 'Morocco' 'Pakistan' None 'Sri Lanka'
 'Central African Republic' 'Albania' 'South Sudan' 'Ukraine'
 'Dem. Rep. of the Congo' 'Guinea' 'Libya' 'China' 'Maldives'
 'Bosnia and Herzegovina' 'Paraguay' 'Burkina Faso' 'El Salvador'
 'Sao Tome & Principe' 'Viet Nam' 'Bhutan' 'Turkmenistan'
 'Solomon Islands' 'Barbados' 'Turkey' 'Burundi' 'Fiji' 'Uzbekistan'
 'Eswatini' 'Kosovo' 'Rwanda' 'Indonesia' 'Mauritius' 'North Macedonia'
 'Equatorial Guinea' 'Armenia' 'Cameroon, Republic of' 'Namibia'

Few approaches I'm thinking

1. Use PandasAI or LangChain to analyze the target DataFrame and provide the results to GPT for text summarization.
2. Convert the DataFrame to a string, give the raw data to GPT for analysis, and request a text summarization.

1. if input is continent/undp_region
   info we need:
   - num of country
   - total budget
   - direct beneficiaries.
   - direct beneficiaries for VF project and Non-VF project
   - direct beneficiaries by Beneficiary Category
2. if input is country
    - total budget
   - direct beneficiaries.
   - direct beneficiaries for VF project and Non-VF project
   - direct beneficiaries by Beneficiary Category

Calculate each info mannually and ask chatGPT to make summary based on mannually calculated value

In [65]:
continent_dict = set(['Africa', 'Americas','Oceania', 'Asia', 'Europe'])
country_dict = set(['Togo', 'Panama', 'Zambia', 'Malawi', 'South Africa', 'Samoa', 'Lebanon', 'Brazil', 'Jamaica', 'Algeria', 'Yemen', 'Belize', 'Iran', 'Colombia', 'Comoros', 
                    'Azerbaijan', 'Moldova Republic', 'Congo', 'Jordan', 'Dominican Republic', 'Sierra Leone', 'Saudi Arabia', 'Senegal', 'Suriname', 'Gambia', 'Uganda', 'Mexico',
                    'Botswana', 'Guyana', 'Ethiopia', 'Belarus', 'Bahrain', 'Papua New Guinea', 'Madagascar', 'Venezuela', 'Lesotho', 'Zimbabwe', 'Ecuador', 'Morocco', 'Pakistan',
                    'Sri Lanka', 'Central African Republic', 'Albania', 'South Sudan', 'Ukraine', 'Dem. Rep. of the Congo', 'Guinea', 'Libya', 'China', 'Maldives', 
                    'Bosnia and Herzegovina', 'Paraguay', 'Burkina Faso', 'El Salvador', 'Sao Tome & Principe', 'Viet Nam', 'Bhutan', 'Turkmenistan', 'Solomon Islands',
                    'Barbados', 'Turkey', 'Burundi', 'Fiji', 'Uzbekistan', 'Eswatini', 'Kosovo', 'Rwanda', 'Indonesia', 'Mauritius', 'North Macedonia', 'Equatorial Guinea',
                    'Armenia', 'Cameroon, Republic of', 'Namibia', 'Chad', 'Cambodia', 'Tunisia', 'Niger', 'Sudan', 'Cuba', 'Benin', 'Kenya', 'Syria', 'Mozambique', 'Costa Rica',
                    'Lao PDR', 'Timor-Leste', 'United Republic of Tanzania', 'Serbia', 'Afghanistan', 'Kyrgyzstan', 'Haiti', 'Prog for Palestinian People', 'Iraq', 'Philippines',
                    'Somalia', 'India', 'Mali', "Cote d'Ivoire", 'Angola', 'Mongolia', 'Thailand', 'Peru', 'Tajikistan', 'Nigeria', 'Georgia', 'Djibouti', 'Trinidad & Tobago',
                    'Bangladesh', 'Myanmar', 'Argentina', 'Kazakhstan', 'Malaysia', 'Guinea-Bissau', 'Montenegro', 'Egypt', 'Mauritania'])
undp_region_dict = set(['RBA', 'RBLAC', 'RBAP', 'RBAS', 'RBEC'])

Example Outputs

RBAP (regional bureau of asia pacific):

The UNDP portfolio of energy-related projects in RBAP includes 83 projects across 23 countries during the 2022-2025 Strategic Plan. The total budget of these projects is 422 million USD. The total direct beneficiaries target by these projects include 16.3 million people. This include 7.8 million from VF projects and 8.5 from Non-VF. 4.3 million of these beneficiaries are from productive use of energy, which includes 1.7 million from health services, 1.2 million from energy infrastructures services, and 400,000 from agricultural and food systems. Many projects also include outputs related to support to policy or regulatory frameworks or other system benefits that provide indirect benefits.

In [120]:
def text_summary_gen(user_input):
    region_type, region_value = get_project_region_type(user_input), user_input
    # ['Project ID', 'Project Title', 'Budget', 'VF or Non-VF', 'Output Count', 'Direct Beneficiaries']
    projects = get_project_per_region(region_type, region_value)
    project_ids = projects['Project ID']
    # ['Project ID', 'Beneficiary Category', 'continent-region', 'Country Name', 'Region', 'Direct Beneficiaries']
    outputs = get_output_per_project(project_ids)

    total_projects = len(projects)
    total_budget = projects['Budget'].sum()
    direct_beneficiaries = projects['Direct Beneficiaries'].sum()
    direct_beneficiaries_VF = projects[projects['VF or Non-VF'] == 'VF']['Direct Beneficiaries'].sum()
    direct_beneficiaries_non_VF = projects[projects['VF or Non-VF'] == 'Non-VF']['Direct Beneficiaries'].sum()
    beneficiary_category = outputs['Beneficiary Category'].unique()
    direct_beneficiaries_by_categories = defaultdict(int)

    for category in beneficiary_category:
        if category:
            direct_beneficiaries_by_categories[category] = outputs[outputs['Beneficiary Category'] == category]['Direct Beneficiaries'].sum()

    if region_type == 'country':
        portfolio_stat_info = {
            'total_projects': total_projects,
            'total_budget': total_budget,
            'direct_beneficiaries': direct_beneficiaries,
            'direct_beneficiaries_VF': direct_beneficiaries_VF,
            'direct_beneficiaries_non_VF': direct_beneficiaries_non_VF,
            'direct_beneficiaries_by_categories': json.dumps(direct_beneficiaries_by_categories)
        }
    else:
        num_of_countries = projects.country.nunique()
        portfolio_stat_info = {
            'total_projects': total_projects,
            'num_of_countries': num_of_countries,
            'total_budget': total_budget,
            'direct_beneficiaries': direct_beneficiaries,
            'direct_beneficiaries_VF': direct_beneficiaries_VF,
            'direct_beneficiaries_non_VF': direct_beneficiaries_non_VF,
            'direct_beneficiaries_by_categories': json.dumps(direct_beneficiaries_by_categories)
        }
        
    llm_prompt = f'''
    This is statistical information regarding the UNDP's project portfolio within the {region_value} area for the 2022-2025 Strategic Plan. Please generate a text summary based on this data.
    Here's an example of the desired summary:
    
    The UNDP portfolio of energy-related projects in RBAP includes 83 projects across 23 countries during the 2022-2025 Strategic Plan. 
    The total budget of these projects is 422 million USD. The total direct beneficiaries target by these projects include 16.3 million people. 
    This include 7.8 million from VF projects and 8.5 from Non-VF. 4.3 million of these beneficiaries are from productive use of energy, 
    which includes 1.7 million from health services, 1.2 million from energy infrastructures services, and 400,000 from agricultural and food systems. 
    Many projects also include outputs related to support to policy or regulatory frameworks or other system benefits that provide indirect benefits.
    
    Here's the provided statistical information: {portfolio_stat_info}

    '''

    answer = callOpenAI(llm_prompt)
    return answer

In [122]:
text_summary_gen('RBA')

"The UNDP's project portfolio within the RBA area for the 2022-2025 Strategic Plan consists of a total of 112 projects across 41 countries. These projects have a combined budget of approximately 1.44 billion USD. \n\nThe projects aim to directly benefit a total of 61.6 million people. Out of these beneficiaries, 6.5 million are targeted by projects classified as VF (Vulnerable Focus), while 55.1 million are targeted by projects classified as non-VF. \n\nThe direct beneficiaries can be further categorized based on the type of benefits they receive. Around 15.8 million people are expected to benefit from projects related to clean electricity, while 19.4 million people will benefit from health services. Additionally, 12.1 million people will benefit from water services, and 1.4 million people will benefit from clean cooking initiatives. \n\nIt is important to note that some projects also aim to provide indirect benefits through support to policy or regulatory frameworks, as well as other 